In [69]:
import sys, os
sys.path.append('../..')
from tools import ML
from tools.instruments import instruments
from tools.display import ohlc_chart, highchart
import h5py
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import time

In [2]:
%load_ext autoreload
%aimport tools.ML
%aimport tools.display
%autoreload 2

### 0. Parameters

In [16]:
window = 50
filepath = os.path.join('..','..','data','trend index','training set', 'trend_50day.hdf')

### 1. Gathering Data

In [76]:
file = h5py.File(filepath, mode='a')

In [65]:
quotes = instruments.quotes(fields='ohlc').copy()
symbols = list(quotes.columns.levels[0])

In [ ]:
file = h5py.File(filepath, mode='a')

num_dim = 4 #open, high, low, close
if 'train' not in file.keys():
        file.create_dataset('train', (0, window, num_dim), maxshape=(None, window, num_dim), dtype='float32', compression='gzip')
        file.create_dataset('trend', (0,), maxshape=(None,), dtype='i')
        file.create_dataset('guess', (0,), maxshape=(None,), dtype='i')
        
Train, Trend, Guess  = file['train'], file['trend'], file['guess']


while True:
    # 상품 랜덤 선택
    symbol = np.random.choice(symbols)
    name = instruments[symbol].name
    quote = quotes[symbol].dropna()
    #print(instruments[symbol].name)
    
    # 10세트 데이터 랜덤 선택
    indexes = np.random.randint(0, len(quote)-window, 10)
    for index in indexes:
        # 데이터 셋 선택 및 규격확
        data = quote.iloc[index:index+window]
        train_data = (data - data.mean())/data.std().mean()
        highchart([{ 'data':data, 'name':name, 'type':'candlestick'}], height=500)
        
        time.sleep(0.1)
        trend = input("추세? 하락(-1), 없음(0), 상승(1): ")
        guess = input("예측? 하락(-1), 관망(0), 상승(1): ")
        if (trend not in ['-1','0','1']) or (guess not in ['-1','0','1']):
            print(f"You pressed trend:{trend} and guess: {guess}")
            break
        else:
            size = Train.shape[0]
            Train.resize(size+1, axis=0)
            Trend.resize(size+1, axis=0)
            Guess.resize(size+1, axis=0)

            Train[size] = data.values
            Trend[size] = int(trend)
            Guess[size] = int(guess)

        clear_output(wait=True)
    
    if (trend not in ['-1','0','1']) or (guess not in ['-1','0','1']):
            break
file.close()

추세? 하락(-1), 없음(0), 상승(1): 0


### 2. Verification

In [7]:
from collections import Counter

In [8]:
file = open_file('h5py', filepath, mode='r')

In [9]:
print(file['X'].shape)
print(file['Y'].shape)

(791, 10, 4)
(791,)


In [10]:
Y = file['Y']
Counter(Y)

Counter({0: 187, 1: 396, 2: 208})

In [11]:
file.close()